In [4]:
# Lo primero que hago para poder trabajar con los datos es importar las librerias necesarias:

# Para tratamiento de datos:
# ----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt


# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind, shapiro, poisson, chisquare, expon, kstest



# configuro pandas para que me muestre todas las columnas del dataframe

pd.set_option('display.max_columns', None)



In [5]:
# ABRO LOS CSV PARA PODER TRABAJAR CON ELLOS

customer_info = pd.read_csv('files/Customer Info Limpio.csv')
customer_final = pd.read_csv('files/Customer Final Limpio.csv')

# hacemos un head de ambos para ver que se han cargado bien

display(customer_info.head(2))
display(customer_final.head(2))

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,73479.0,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN


,Loyalty Number,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,480934,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016.0,2.0,NaN,NaN,2017.0,1.0,0,0,0,0,0.0,0,0
1,480934,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016.0,2.0,NaN,NaN,2017.0,2.0,3,0,3,2823,282.0,0,0


## FASE 3 (BONUS) : Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

### 3.1: Preparación de los Datos

In [6]:
# Vamos a tener un grupo por cada nivel educativo, sobre el que calcularemos en numero de reservas de vuelos y haremos las comparativas

print(f"""Los diferentes niveles educativos que hay son {customer_final['Education'].unique()}""")

Los diferentes niveles educativos que hay son ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']


In [7]:

lista_education = customer_final['Education'].unique() 

print(f"""
Saco una lista con todas las diferentes categorias de educacion que hay en el dataframe
lista_education ==> {lista_education}""") 

# Hago un bucle for para modificar la lista y cada elemento que sea un dataframe con la columnas que queremos seleccionar, 
# para ello itero sobre lista_education y modifico cada index de esta nueva lista (al que accedo con el contador) para que 
# en cada elemento tengamos un df con los datos buscados, y poder así hacer los grupos.

contador = 0
for i in lista_education:
    lista_education[contador] = customer_final[customer_final['Education'] == i][['Education', 'Flights Booked']]
    contador += 1

grupo_bachelor = lista_education[0]
grupo_college = lista_education[1]
grupo_master = lista_education[2]
grupo_highschool = lista_education[3]
grupo_doctor = lista_education [4]

print(f"Los grupos que tenemos son {len(lista_education)}, que son:\n")

print("BACHELOR ==>")
display(grupo_bachelor.head(3))

print("COLLEGE ==>")
display(grupo_college.head(3))

print("MASTER ==>")
display(grupo_master.head(3))

print("HIGH SCHOOL ==>")
display(grupo_highschool.head(3))

print("DOCTOR ==>")
display(grupo_doctor.head(3))


# Con este bucle for tengo una lista 



Saco una lista con todas las diferentes categorias de educacion que hay en el dataframe
lista_education ==> ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
Los grupos que tenemos son 5, que son:

BACHELOR ==>


,Education,Flights Booked
0,Bachelor,0
1,Bachelor,3
2,Bachelor,0


COLLEGE ==>


,Education,Flights Booked
24,College,9
25,College,9
26,College,14


MASTER ==>


,Education,Flights Booked
381,Master,2
382,Master,12
383,Master,0


HIGH SCHOOL ==>


,Education,Flights Booked
813,High School or Below,9
814,High School or Below,13
815,High School or Below,2


DOCTOR ==>


,Education,Flights Booked
1341,Doctor,7
1342,Doctor,0
1343,Doctor,15


### 3.2: Analisis Descriptivo

In [8]:
print("""Vamos a calcular algunas medidas estadísticas descriptivas para cada grupo
""")


display(grupo_bachelor.groupby('Education')['Flights Booked'].agg(['mean', 'median', 'sum', 'std']))
print("-----------------------------------------------------------------------------------------")
display(grupo_college.groupby('Education')['Flights Booked'].agg(['mean', 'median', 'sum', 'std']))
print("-----------------------------------------------------------------------------------------")
display(grupo_master.groupby('Education')['Flights Booked'].agg(['mean', 'median', 'sum', 'std']))
print("-----------------------------------------------------------------------------------------")
display(grupo_highschool.groupby('Education')['Flights Booked'].agg(['mean', 'median', 'sum', 'std']))
print("-----------------------------------------------------------------------------------------")
display(grupo_doctor.groupby('Education')['Flights Booked'].agg(['mean', 'median', 'sum', 'std']))

Vamos a calcular algunas medidas estadísticas descriptivas para cada grupo



,mean,median,sum,std
Education,,,,
Bachelor,4.110288,1.0,1038123,5.221671


-----------------------------------------------------------------------------------------


,mean,median,sum,std
Education,,,,
College,4.169744,1.0,426398,5.24604


-----------------------------------------------------------------------------------------


,mean,median,sum,std
Education,,,,
Master,4.2007,1.0,51614,5.213956


-----------------------------------------------------------------------------------------


,mean,median,sum,std
Education,,,,
High School or Below,4.176209,1.0,78993,5.239267


-----------------------------------------------------------------------------------------


,mean,median,sum,std
Education,,,,
Doctor,4.175512,1.0,74036,5.256971


In [9]:
print(f"""
Aparentemente, echando un vistado a las medidas estadisticas descriptivas no hay una diferencia significativa ni en media de 
vuelos alquilados por grupos, la mediana es igual tambien para todos los grupos, y la desviacion tipica igualmente es similar. 
El grupo que claramente destaca atendiendo al numero de vuelos totales es bachelor, seguido de college, posiblemente porque sean los grupos
a los que mayoritariamente pertenecen nuestros clientes
{customer_info.groupby('Education')['Loyalty Number'].count()}
Efectivamente, comprobamos que predominan claramente los clientes con educacion Bachelor y College""")


Aparentemente, echando un vistado a las medidas estadisticas descriptivas no hay una diferencia significativa ni en media de 
vuelos alquilados por grupos, la mediana es igual tambien para todos los grupos, y la desviacion tipica igualmente es similar. 
El grupo que claramente destaca atendiendo al numero de vuelos totales es bachelor, seguido de college, posiblemente porque sean los grupos
a los que mayoritariamente pertenecen nuestros clientes
Education
Bachelor                10475
College                  4238
Doctor                    734
High School or Below      782
Master                    508
Name: Loyalty Number, dtype: int64
Efectivamente, comprobamos que predominan claramente los clientes con educacion Bachelor y College


### 3.3: Prueba Estadistica

In [10]:
print("""
Primero definimos las hipótesis:
- H0: No hay diferencia significativa entre grupos
- H1: Sí hay una diferencia significativa entre grupos
""")


Primero definimos las hipótesis:
- H0: No hay diferencia significativa entre grupos
- H1: Sí hay una diferencia significativa entre grupos



In [11]:
print("""
Lo siguiente que haremos es ver si sigue una distribución normal o no, para lo que usaremos el test de saphiro
""")


Lo siguiente que haremos es ver si sigue una distribución normal o no, para lo que usaremos el test de saphiro



In [12]:
# Uno los dataframe para tener un unico dataframe con la info solo de las columnas Education y Flights Booked
df_grupos_education = pd.concat([grupo_bachelor,grupo_college,grupo_doctor,grupo_highschool,grupo_master])
df_grupos_education.sample(5)

,Education,Flights Booked
402104,Bachelor,0
349042,Bachelor,13
116398,Bachelor,7
330048,Bachelor,5
280960,College,17


### NORMALIDAD

In [13]:
# Primero vamos a valorar si los datos siguen una distribución normal. 
# Lo haremos con el test estadístico shapiro
p_value = shapiro(df_grupos_education['Flights Booked']).pvalue

alpha = 0.05

if p_value > alpha:
    print("Los datos se ajustan a una distribución normal (p-value =", p_value, ")")
else:
    print("Los datos no se ajustan a una distribución normal (p-value =", p_value, ")")

# En este caso sale un aviso de visual studio code diciendo que para grandes muestras es mejor usar otro test. 

Los datos no se ajustan a una distribución normal (p-value = 1.5469185712625606e-164 )


C:\Users\Laura Romero\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_axis_nan_policy.py:573: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 403760.
  res = hypotest_fun_out(*samples, **kwds)


In [14]:
# Por si acaso, repito el test de normalidad, pero en este caso usando el test de Kolmogorov
p_value = kstest(df_grupos_education['Flights Booked'], "norm").pvalue

alpha = 0.05
if p_value > alpha:
    print("Los datos se ajustan a una distribución normal (p-value =", p_value, ")")
else:
    print("Los datos no se ajustan a una distribución normal (p-value =", p_value, ")")

Los datos no se ajustan a una distribución normal (p-value = 0.0 )


### HOMOGENEIDAD VARIANZAS

In [22]:
# El siguiente paso es analizar la homogeneidad de las varianzas
# Como estamos ante una distribución no normal usamos Levene. 

p_valor_varianza = stats.levene(grupo_bachelor['Flights Booked'],grupo_college['Flights Booked'],grupo_doctor['Flights Booked'],grupo_highschool['Flights Booked'],grupo_master['Flights Booked'], center = "median")[1]

print(f"p_value = {p_valor_varianza}")
if p_value > 0.05:
    print(f"No hay diferencia significativa entre las varianzas de los grupos")
else:
    print(f"Si hay diferencias significativas entre las varianzas de los grupos")

p_value = 0.014969793534581529
No hay diferencia significativa entre las varianzas de los grupos


### PRUEBA NO PARAMÉTRICA

In [ ]:
print("""
Como hay dos supuestos que no se cumplen, que son normalidad y homogeneidad de varianzas, tendremos que usar para verificar o descartar nuestra hipótesis
pruebas de tipo no paramétrico, en este caso utilizaré el test de homogeneidad de medianas.
""")

In [27]:
# Para hacer el estudio tengo dos opciones, o comparar cada grupo de dos a dos, para ver la relación entre los diferentes grupos. Pues no existe ningún test 
# que pueda hacer el estudio con los cinco grupos simultáneamente. 

grupos = [grupo_bachelor, grupo_master, grupo_doctor, grupo_highschool, grupo_college] # hago una lista con los 5 dataframes que contienen la informacion de cada grupo
medianas_diferentes = [] # hago dos listas vacias para ir introduciendo los grupos con medianas distintas y iguales
medianas_iguales = []
grupo_ya_testeado = [] # para no repetir comparación (ejemplo: si tengo Bachelor-Master, no repetirlo a la inversa Master-Bachelor)

for control in grupos:
    grupo_ya_testeado.append(control['Education'].iloc[0]) 
    # Para que una vez que se evalue un grupo con otro, no se repita de nuevo al reves: ejemplo, si se ha visto que bachelor y master tienen medianas diferentes, 
    # siendo en ese caso Bachelor control y Master test, que cuando el control sea Master no vuelva a hacerse la comparacion con Bachelor, pues ya esta hecha. 
    for test in grupos:
        if test['Education'].iloc[0] != control['Education'].iloc[0]: # para que no evalue mismo grupo con mismo grupo
            if test['Education'].iloc[0] not in grupo_ya_testeado:
                # aplicamos el estadístico de MannWhitney
                u_statistic, p_value = stats.mannwhitneyu(control['Flights Booked'], test['Flights Booked'])
                if p_value < 0.05:
                    print(f"Para la métrica Flights Booked, las medianas son diferentes entre {control['Education'].iloc[0]} y {test['Education'].iloc[0]}")
                    medianas_diferentes.append(f"{control['Education'].iloc[0]} y {test['Education'].iloc[0]}")
                else:
                    print(f"Para la métrica Flights Booked, las medianas son iguales entre {control['Education'].iloc[0]} y {test['Education'].iloc[0]}")
                    medianas_iguales.append(f"{control['Education'].iloc[0]} y {test['Education'].iloc[0]}")

Para la métrica Flights Booked, las medianas son diferentes entre Bachelor y Master
Para la métrica Flights Booked, las medianas son iguales entre Bachelor y Doctor
Para la métrica Flights Booked, las medianas son diferentes entre Bachelor y High School or Below
Para la métrica Flights Booked, las medianas son diferentes entre Bachelor y College
Para la métrica Flights Booked, las medianas son iguales entre Master y Doctor
Para la métrica Flights Booked, las medianas son iguales entre Master y High School or Below
Para la métrica Flights Booked, las medianas son iguales entre Master y College
Para la métrica Flights Booked, las medianas son iguales entre Doctor y High School or Below
Para la métrica Flights Booked, las medianas son iguales entre Doctor y College
Para la métrica Flights Booked, las medianas son iguales entre High School or Below y College


In [26]:
print(f"""
El resultado de comparar grupos dos a dos ha dado que las medianas son diferentes para ==> 
{medianas_diferentes}
Numero de grupos discrepantes entre sí ==> {len(medianas_diferentes)}
----------------------------------------------------------------------------------------------------------------------------------------
y con iguales para
{medianas_iguales}
Numero de grupos similares entre sí ==> {len(medianas_iguales)}
-----------------------------------------------------------------------------------------------------------------------------------------
Las medianas son iguales para todos los grupos excepto en el caso de Bachelor, que solo tiene similitudes con Doctor, pero tiene diferencias
significativas con el resto. 
Podríamos decir que el grupo menos similar al resto es Bachelor""")


El resultado de comparar grupos dos a dos ha dado que las medianas son diferentes para ==> 
['Bachelor y Master', 'Bachelor y High School or Below', 'Bachelor y College']
Numero de grupos discrepantes entre sí ==> 3
----------------------------------------------------------------------------------------------------------------------------------------
y con iguales para
['Bachelor y Doctor', 'Master y Doctor', 'Master y High School or Below', 'Master y College', 'Doctor y High School or Below', 'Doctor y College', 'High School or Below y College']
Numero de grupos similares entre sí ==> 7
-----------------------------------------------------------------------------------------------------------------------------------------
Las medianas son iguales para todos los grupos excepto en el caso de Bachelor, que solo tiene similitudes con Doctor, pero tiene diferencias
significativas con el resto. 
Podríamos decir que el grupo menos similar al resto es Bachelor


In [29]:
# Podemos hacerlo mirando si hay diferencia significativa entre la reserva de vuelos entre los grupos con mayores ingresos (Doctor y Master)
# y los grupos con menor ingresos, pues los resultados de grupos evaluados a pares no son demasiado concluyentes. 

customer_info.groupby('Education')['Salary'].mean()

# en este caso, como ya he evaluado normalidad y homogeneidad de varianzas podría usar directamente el estadistico Mann Whitney para estos dos grupos.

grupo_altos_ingresos =  

Education
Bachelor                 72451.061575
College                  73479.000000
Doctor                  178608.897820
High School or Below     61071.718670
Master                  103757.848425
Name: Salary, dtype: float64

In [38]:
# Otra forma de hacerlo, que sería a la vez interesante, en lugar de evaluar los 5 grupos, evaluar los grupos con altos ingresos ()

# grupo altos ingresos, filtro el dataframe para que contenga las columnas Education == Master o Education == Doctor
grupo_altos_ingresos = customer_final[(customer_final['Education'] == 'Master')|(customer_final['Education'] == 'Doctor')][['Education', 'Flights Booked']]

# Filtro igual que en el caso anterior, pero para que el dataframe bajos ingresos no contenga ni Master ni Doctor en la columna Education
grupo_bajos_ingresos = customer_final[(customer_final['Education'] != 'Master')&(customer_final['Education'] != 'Doctor')][['Education', 'Flights Booked']]

print(f"""
El grupo de altos ingresos lo forma ==> {grupo_altos_ingresos['Education'].unique()}
El grupo de bajos ingresos lo forma ==> {grupo_bajos_ingresos['Education'].unique()}
      """)

print("Echamos un vistazo a ambos dataframes para ver que contienen la informacion que nos interesa, Education y Flights Booked")

display(grupo_altos_ingresos.sample(4))
display(grupo_bajos_ingresos.sample(4))


El grupo de altos ingresos lo forma ==> ['Master' 'Doctor']
El grupo de bajos ingresos lo forma ==> ['Bachelor' 'College' 'High School or Below']
      
Echamos un vistazo a ambos dataframes para ver que contienen la informacion que nos interesa, Education y Flights Booked


,Education,Flights Booked
4707,Master,2
33838,Doctor,0
114000,Master,10
273217,Doctor,0


,Education,Flights Booked
181590,Bachelor,0
300664,Bachelor,0
104367,Bachelor,11
9712,Bachelor,12


In [40]:
# Sobre estos dos grupos aplicariamos directamente el test de MannWhitney

u_statistic, p_value_2 = stats.mannwhitneyu(grupo_altos_ingresos['Flights Booked'], grupo_bajos_ingresos['Flights Booked'])

print(f"p_value = {p_value_2}")

if p_value_2 < 0.05:
    
    print(f"Para la métrica Flights Booked, las medianas son diferentes entre {grupo_bajos_ingresos['Education'].unique()} y {grupo_altos_ingresos['Education'].unique()}")
    
else:
    print(f"Para la métrica Flights Booked, las medianas son iguales entre {grupo_bajos_ingresos['Education'].unique()} y {grupo_altos_ingresos['Education'].unique()}")




p_value = 0.029749016772146173
Para la métrica Flights Booked, las medianas son diferentes entre ['Bachelor' 'College' 'High School or Below'] y ['Master' 'Doctor']
